# #15 atmaCup

---

## セットアップ

In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
import lightgbm as lgb

In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
anime = pd.read_csv('data/anime.csv', na_values=['Unknown'])

## 前処理

### 共通の前処理

In [ ]:
# 作品名からIPを抽出する。
# データ観察の結果から、作品名の先頭4文字でIPを区別できると仮定している。
anime['ip'] = (
    anime['japanese_name']
    .fillna('')
    # 作品名の先頭に '劇場版' が付く場合が多いため、あらかじめ '劇場版' を除いておく。
    .str.replace('劇場版', '', regex=False)
    .str.strip()
    .str.lower()
    .str.normalize('NFKC')
    .str.extract(r'^([^\s]+)')[0]
    .str.slice(0, 4)
)

In [ ]:
train_anime = train.merge(anime, how='left', on='anime_id')
test_anime = test.merge(anime, how='left', on='anime_id')
traintest_anime = pd.concat([train_anime, test_anime], ignore_index=True)

### ユーザーとカテゴリーの分散表現の獲得

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
import scipy as sp

# 評価行列 (行：ユーザー、列：カテゴリー、値：評価件数など) を行列分解して
# ユーザーとカテゴリー (アニメ作品、ジャンル、……) の分散表現を作る。
class UserCategoryNMFTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, traintest_anime, category_column, value_column, func, sep, n_components):
        # 評価行列の元になるデータ。
        # 学習データ (train.csv) とテストデータ (test.csv) の両方から分散表現を作りたいので、
        # 両者を結合したデータを指定する。
        self.traintest_anime = traintest_anime
        # 評価行列の列を表すカラム。
        self.category_column = category_column
        # 評価行列の値を表すカラム。
        # Noneを指定した場合、評価行列の値として評価件数が使われる。
        self.value_column = value_column
        # 評価行列の値の集計に用いる関数。
        self.func = func
        # 複数カテゴリー変数を分割する際に用いるセパレーター。
        # 分割せず単カテゴリー変数として扱ったほうが精度が良くなる変数があったことから、
        # 分割するか否かを決めるために用いる。
        self.sep = sep
        # 分散表現の次元数。
        self.n_components = n_components

    def fit(self, X, y=None):
        # ユーザー数 x カテゴリー (アニメ作品、ジャンル、……) のスパース行列を作る。
        # 行：ユーザー (user_id)。
        # 列：カテゴリー (アニメ作品、ジャンル、……)。
        # 値：その行のユーザーのその列のカテゴリーに対する評価件数など。
        le_user = LabelEncoder()
        le_category = LabelEncoder()
        count = (
            self.traintest_anime
            .assign(
                category=lambda df: df[self.category_column].str.split(self.sep),
                value=lambda df: 1 if self.value_column is None else np.log1p(df[self.value_column])
            )
            .loc[:, ['user_id', 'category', 'value']]
            .explode('category')
            .groupby(['user_id', 'category'])['value'].agg(self.func)
            .rename('count')
            .reset_index()
            .assign(
                user_id=lambda df: le_user.fit_transform(df['user_id']),
                category=lambda df: le_category.fit_transform(df['category'])
            )
        )
        user_category_matrix = csr_matrix((count['count'], (count['user_id'], count['category'])))

        # ユーザーの分散表現
        # shape: (ユーザー数, n_components)
        nmf = NMF(n_components=self.n_components, alpha_W=0.01, max_iter=1000, random_state=0)
        user_nmf = nmf.fit_transform(user_category_matrix)
        user_nmf = pd.DataFrame(
            user_nmf,
            index=pd.Index(le_user.classes_, name='user_id'),
            columns=[f'user_nmf_{i:02d}' for i in range(self.n_components)]
        )
        self.user_nmf = user_nmf

        # カテゴリーの分散表現
        # shape: (カテゴリー数, n_components)
        category_nmf = nmf.components_.T
        category_nmf = pd.DataFrame(
            category_nmf,
            index=pd.Index(le_category.classes_, name='category'),
            columns=[f'category_nmf_{i:02d}' for i in range(self.n_components)]
        )
        self.category_nmf = category_nmf

        return self

    def transform(self, X):
        target_columns = list(set(['user_id', 'anime_id', self.category_column]))
        X_new = (
            X[target_columns]
            .merge(self.user_nmf, how='left', on='user_id')
            # カテゴリーの分散表現を紐づける。
            # 該当するカテゴリーが複数ある場合 (ジャンルなど) は、
            # 各ジャンルの分散表現の和を紐づける。
            .merge(
                right=(
                    X[target_columns]
                    .assign(category=lambda df: df[self.category_column].str.split(self.sep))
                    .explode('category')
                    .merge(self.category_nmf, how='left', on='category')
                    .groupby(['user_id', 'anime_id']).sum(numeric_only=True)
                ),
                how='left',
                on=['user_id', 'anime_id']
            )
            .drop(columns=target_columns)
            .to_numpy()
        )

        return X_new

    def get_feature_names_out(self, input_features=None):
        names = np.concatenate([
            self.user_nmf.columns.to_numpy(),
            self.category_nmf.columns.to_numpy()
        ])

        return names

### 評価件数の集計

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# 各ユーザーが評価した作品数をカウントする。
class UserCounter(BaseEstimator, TransformerMixin):
    def __init__(self, traintest_anime):
        self.traintest_anime = traintest_anime

    def fit(self, X, y=None):
        # 各ユーザーの総評価件数
        # shape: (ユーザー数, 1)
        self.user_count = (
            self.traintest_anime
            .groupby('user_id').size()
            .rename('user')
            .to_frame()
        )

        # 各ユーザーのジャンル別の評価件数
        # shape: (ユーザー数, ジャンル数)
        self.user_genre_count = (
            self.traintest_anime
            .assign(genres=lambda df: df['genres'].str.split(', '))
            .explode('genres')
            .groupby(['user_id', 'genres']).size()
            .rename('count')
            .reset_index()
            .pivot(index='user_id', columns='genres', values='count')
            .fillna(0.0)
        )
        self.user_genre_count.columns = 'genre_' + self.user_genre_count.columns

        # 各ユーザーのIP別の評価件数
        # shape: (ユーザー数 * IP数, 1)
        self.user_ip_count = (
            self.traintest_anime
            .groupby(['user_id', 'ip']).size()
            .rename('count_ip')
            .to_frame()
        )

        return self

    def transform(self, X):
        X_new = (
            X[['user_id', 'anime_id', 'genres', 'ip']]
            .merge(self.user_count, how='left', on='user_id')
            .merge(self.user_genre_count, how='left', on='user_id')
            .merge(self.user_ip_count, how='left', on=['user_id', 'ip'])
            .drop(columns=['user_id', 'anime_id', 'genres', 'ip'])
            .to_numpy()
        )

        return X_new

    def get_feature_names_out(self, input_features=None):
        names = np.concatenate([
            self.user_count.columns.to_numpy(),
            self.user_genre_count.columns.to_numpy(),
            self.user_ip_count.columns.to_numpy()
        ])

        return names

### その他の特徴抽出

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class AssortedTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_new = X.copy()

        # カテゴリー変数をカテゴリー型に変換する (LightGBMの categorical_feature に指定する)。
        for column in ['type', 'source', 'rating']:
            X_new[column] = X_new[column].astype('category')

        # airedから放送開始年を抽出する。
        X_new['aired'] = X['aired'].str.extract('(\d{4})').astype(float)

        # durationから時間と分を抽出して分に変換する。
        X_new['duration'] = (
            X['duration'].str.extract(r'(\d+) hr.').astype(float).fillna(0.0) * 60 +
            X['duration'].str.extract(r'(\d+) min.').astype(float).fillna(0.0)
        )

        # 見るのを中断した人数と最後まで見た人数の比
        X_new['dpr'] = X['dropped'] / X['plan_to_watch']

        return X_new

    def get_feature_names_out(self, input_features=None):
        names = feature_names_in_ + ['dpr']

        return names

### 特徴抽出のパイプラインの構築

In [ ]:
pipeline = ColumnTransformer(
    transformers=[
        (
            f'{transformer_name}' if value_column is None else f'{transformer_name}_{value_column}',
            UserCategoryNMFTransformer(
                traintest_anime=traintest_anime,
                category_column=category_column,
                value_column=value_column,
                func=func,
                sep=sep,
                n_components=n_components
            ),
            list(set(['user_id', 'anime_id', category_column]))
        )
        for transformer_name, category_column, sep, n_components in [
            ('nmf_anime_id_1', 'anime_id', '__dummy_sep__', 50),
            ('nmf_anime_id_2', 'anime_id', '__dummy_sep__', 100),
            # ('nmf_anime_id_3', 'anime_id', '__dummy_sep__', 75),
            ('nmf_genres_1', 'genres', ', ', 5),
            ('nmf_genres_2', 'genres', ', ', 10),
            # ('nmf_genres_3', 'genres', ', ', 30),
            ('nmf_producers_1', 'producers', ', ', 50),
            # ('nmf_producers_1', 'producers', '__dummy_sep__', 50),
            ('nmf_producers_2', 'producers', ', ', 100),
            # ('nmf_producers_2', 'producers', '__dummy_sep__', 100),
            # ('nmf_producers_3', 'producers', ', ', 75),
            # ('nmf_producers_3', 'producers', '__dummy_sep__', 75),
            ('nmf_licensors_1', 'licensors', ', ', 5),
            # ('nmf_licensors_1', 'licensors', '__dummy_sep__', 5),
            ('nmf_licensors_2', 'licensors', ', ', 10),
            # ('nmf_licensors_2', 'licensors', '__dummy_sep__', 10),
            # ('nmf_licensors_3', 'licensors', ', ', 30),
            # ('nmf_licensors_3', 'licensors', '__dummy_sep__', 30),
            ('nmf_studios_1', 'studios', ', ', 20),
            # ('nmf_studios_1', 'studios', '__dummy_sep__', 20),
            ('nmf_studios_2', 'studios', ', ', 40),
            # ('nmf_studios_2', 'studios', '__dummy_sep__', 40),
            # ('nmf_studios_3', 'studios', ', ', 30),
            # ('nmf_studios_3', 'studios', '__dummy_sep__', 30),
            ('nmf_source_1', 'source', '__dummy_sep__', 5),
            ('nmf_source_2', 'source', '__dummy_sep__', 10),
            # ('nmf_source_3', 'source', '__dummy_sep__', 30),
            ('nmf_ip_1', 'ip', '__dummy_sep__', 30),
            ('nmf_ip_2', 'ip', '__dummy_sep__', 60),
            # ('nmf_ip_3', 'ip', '__dummy_sep__', 45),
        ]
        for value_column, func in [
            (None, 'sum'),
            # ('plan_to_watch', 'mean'),
            # ('dropped', 'mean')
        ]
    ] + [
        (
            'count',
            UserCounter(traintest_anime=traintest_anime),
            ['user_id', 'anime_id', 'genres', 'ip']
        ),
        (
            'other',
            AssortedTransformer(),
            [
                'type',
                'episodes',
                'aired',
                'source',
                'duration',
                'rating',
                'members',
                'watching',
                'completed',
                'on_hold',
                'dropped',
                'plan_to_watch'
            ]
        )
    ],
    verbose=True
)
pipeline = pipeline.set_output(transform='pandas')

## 学習

In [ ]:
from sklearn.model_selection import KFold, GroupKFold

# テストデータと同じように学習データに存在しないユーザーの評価結果が約23％を占めるようデータを分割する。
class UnknownUserKFold:
    def __init__(self, n_splits_cv, n_splits_uu):
        self.n_splits_cv = n_splits_cv
        self.n_splits_uu = n_splits_uu

    def split(self, X, y=None, groups=None):
        splits_cv = KFold(n_splits=self.n_splits_cv, shuffle=True, random_state=0).split(X)
        splits_uu = GroupKFold(n_splits=self.n_splits_uu).split(X, groups=groups)
        for fold in range(self.n_splits_cv):
            train_index, test_index = next(splits_cv)
            _, uu_index = next(splits_uu)
            train_index = np.setdiff1d(train_index, uu_index)
            test_index = np.union1d(test_index, uu_index)

            yield train_index, test_index

In [ ]:
x_train = pipeline.fit_transform(train_anime, train_anime['score'])
y_train = train['score']
user_id_train = train['user_id']

models = []
scores = []
cv_details = []
kf = UnknownUserKFold(n_splits_cv=5, n_splits_uu=18)
for i, (train_index, test_index) in enumerate(kf.split(x_train, groups=user_id_train)):
    cv_x_train = x_train.iloc[train_index, :]
    cv_y_train = y_train.iloc[train_index]
    cv_x_test = x_train.iloc[test_index, :]
    cv_y_test = y_train.iloc[test_index]

    model = lgb.train(
        params={
            'objective': 'regression',
            'verbose': -1,
            'metric': 'rmse',
            'learning_rate': 0.01,
            'num_leaves': 100,
            'feature_fraction': 0.7,
            'seed': 127
        },
        train_set=lgb.Dataset(cv_x_train, label=cv_y_train),
        valid_sets=[lgb.Dataset(cv_x_test, label=cv_y_test)],
        num_boost_round=20000,
        callbacks=[lgb.early_stopping(stopping_rounds=500, verbose=True)]
    )
    models.append(model)
    scores.append(model.best_score['valid_0']['rmse'])
    print('')

    cv_details_ = train_anime.iloc[test_index].copy()
    cv_details_['score_pred'] = model.predict(cv_x_test)
    cv_details_['score'] = cv_y_test
    cv_details.append(cv_details_)

cv_details = pd.concat(cv_details, ignore_index=True)
print(f'cv: {np.mean(scores):.4f} ± {np.std(scores):.4f}')

## 予測

In [ ]:
x_test = pipeline.transform(test_anime)
y_test_pred = np.mean([model.predict(x_test) for model in models], axis=0)

In [ ]:
sub = pd.DataFrame(data={'score': y_test_pred})
sub.loc[sub['score'] < 1, 'score'] = 1
sub.loc[sub['score'] > 10, 'score'] = 10
sub.to_csv('submission.csv', index=False, header=True)